In [49]:
import torch
from torch import nn
import numpy as np
import copy
import torch.optim as optim

In [50]:
file_name_model_latest_version = 'Model/model_latest_version.pt'

#device = torch.device("cuda")

In [51]:
text = ['hey how are you','good i am fine','have a nice day', 'sure I will do it']

chars = set(''.join(text))
int2char = dict(enumerate(chars))

char2int = {char: ind for ind, char in int2char.items()}

max_len = len(max(text, key = len))
len_of_text = len(text)
for i in range(0, len(text)):
    text[i] += ' ' * (max_len - len(text[i]))
    print(text[i])

hey how are you  
good i am fine   
have a nice day  
sure I will do it


In [52]:
input_data_len = 5
len_of_dic = len(char2int)
array_template = [[0] * len_of_dic] * input_data_len
array_template

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [53]:
#to build a batch size

input_data_len = 5
len_of_dic = len(char2int)
input_data = []
output_data = []

for no_of_text in range(0, len_of_text):
    for i in range(0, max_len - input_data_len):
        input_data.append(text[no_of_text][i : i + input_data_len])
        output_data.append(text[no_of_text][i + 1 : i + input_data_len + 1])

size_input_data = len(input_data)
array_template = [[0 for i in range(len_of_dic)] for j in range(input_data_len)]
converted_input_data = []
converted_output_data = []

for m in range(0, size_input_data):
    new_input_data = copy.deepcopy(array_template)
    new_output_data = copy.deepcopy(array_template)
    for n in range(0, input_data_len):
        new_input_data[n][char2int[input_data[m][n]]] = 1
        new_output_data[n][char2int[output_data[m][n]]] = 1
    #print(new_input_data)
    converted_input_data.append(copy.deepcopy(new_input_data))
    converted_output_data.append(copy.deepcopy(new_output_data))

input_data_tensor = torch.Tensor(converted_input_data)
output_data_tensor = torch.Tensor(converted_output_data)

In [54]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [55]:
def training_model():
    learning_rate = 0.3
    epoch_size = 50000
    steps_for_printing_out_loss = 5000
    
    RNN_model = Model(input_size = len_of_dic, output_size = len_of_dic, hidden_dim = 16, n_layers = 1)
    
    #RNN_model.to(device)
    loss_functioin = nn.MSELoss()
    optimizer = optim.SGD(RNN_model.parameters(), lr = learning_rate)
    input = input_data_tensor
    target = output_data_tensor

    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output, hidden = RNN_model(input)
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1)))
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        optimizer.step()

    torch.save({'state_dict': RNN_model.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)

In [56]:
def RNN_model_prediction():
    model_latest_version = Model(input_size = len_of_dic, output_size = len_of_dic, hidden_dim = 16, n_layers = 1)
    #model_latest_version.to(device)
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    training_data = input_data_tensor
    model_prediction, hidden_output = model_latest_version(training_data)
    print(model_prediction.size())
    for k in model_prediction:
        next_letter_number = k.argmax()
        next_letter_number = int(next_letter_number.cpu().numpy())
        next_letter = int2char[next_letter_number]
        print(next_letter)

In [57]:
training_model()
RNN_model_prediction()

Loss (epoch: 5000): 0.021613544
Loss (epoch: 10000): 0.019638924
Loss (epoch: 15000): 0.018617999
Loss (epoch: 20000): 0.017845502
Loss (epoch: 25000): 0.017123086
Loss (epoch: 30000): 0.016458588
Loss (epoch: 35000): 0.015881075
Loss (epoch: 40000): 0.015372951
Loss (epoch: 45000): 0.01493763
Loss (epoch: 50000): 0.014587286
torch.Size([240, 21])
o
 
 
a
o
 
 
 
o
w
 
u
o
w
 
a
o
w
 
a
o
w
 
a
r
o
 
a
r
e
 
a
r
e
 
a
 
e
 
y
 
e
 
y
o
e
 
d
o
u
 
a
o
u
 
a
o
u
 
 
o
o
d
 
i
o
d
 
i
 
o
 
i
 
a
 
i
 
a
r
a
 
a
r
 
 
a
r
 
f
a
 
 
f
i
 
 
f
i
n
 
f
i
n
e
a
i
n
e
 
i
n
e
 
 
 
e
 
 
 
o
r
e
 
a
 
e
 
a
 
e
 
a
 
n
 
a
 
n
i
a
 
n
i
c
 
n
i
c
e
a
i
c
e
 
i
c
e
 
d
 
e
 
d
a
e
 
d
a
y
 
a
a
y
 
a
o
y
 
 
u
r
e
 
y
r
e
 
y
 
e
 
d
 
w
 
a
 
w
i
a
 
w
i
l
 
w
i
l
l
a
i
l
l
 
 
l
l
 
d
 
l
 
d
o
 
 
d
o
 
 
d
o
 
i
a
o
 
i
 


In [ ]:
# continue to train the model till a acceptable accuracy
# create a prediction function, to base on a word, to predict next characters
# check the hidden result
# try with more than 1 RNN layer

In [58]:
input_data[0][2]
char2int[input_data[0][2]]
new_input_data[0][0]

0

In [59]:
char2int["c"]
input_data
array_template = [[0] * len_of_dic] * input_data_len
array_template[0, 0]=1
array_template

TypeError: list indices must be integers or slices, not tuple

In [ ]:
len_input_data = len(input_data)
array_template = [[0] * input_data_len] * len_input_data
array_template

In [ ]:
text